In [2]:
# The Main Function for APAD Project -- Cloth On Fly

# Author: Tianyi (Kelly) Zhang
#         Shikha Singh

# Creation Date: 07/21/2019

# Major: MSITM

In [3]:
import sqlite3
import glob
from datetime import date
from datetime import timedelta

### Databate Create

In [4]:
db = sqlite3.connect("ClothONFly.db")
cursor = db.cursor()

In [5]:
# cursor.execute('''
#               CREATE TABLE Users(User_ID INTEGER PRIMARY KEY AutoIncrement, User_Name Text NOT NULL UNIQUE, Password Text,
#               User_Type Text Not Null check(User_Type = "Admin" or User_Type = "Buyer" or User_Type = "Seller"), First_Name TEXT NOT NULL,
#               Last_Name TEXT NOT NULL, Email TEXT Not NUll, Address Text Not NUll, Phone_Num INTEGER Not Null)
#             ''')

# cursor.execute('''
#               CREATE TABLE Inventory_Items(Item_ID INTEGER PRIMARY KEY AutoIncrement, Brand_Name TEXT NOT NULL,
#               Cloth_Type TEXT NOT NULL, Size TEXT NOT NULL, Gender TEXT NOT NULL, Original_Price REAL NOT NULL,
#               Rental_Price REAL NOT NULL, Available_From DATE, Owner_ID INTEGER, Location TEXT NOT NULL, Cloth_Image BLOB NOT NULL,
#               Foreign Key(Owner_ID) References Users(User_ID))
#             ''')

# cursor.execute('''
#               CREATE TABLE Orders (Order_ID INTEGER PRIMARY KEY AutoIncrement, User_ID INTEGER NOT NULL, Item_ID INTEGER Not Null,
#               Delivery_Date Date Not Null, Return_Date Date Not Null, Shipping_Address Text Not Null,
#               Order_Status Text Not Null check(Order_Status = "Order Received(Shipment Pending)" or Order_Status = "Order Shipped" or Order_Status = "Return Initiated" or Order_Status = "Return Received(Deposit Refunded)" or Order_Status = "Return Not Received(Deposit Withheld)"),
#               Shipping_Date Date, FOREIGN KEY(User_ID) REFERENCES Users(User_ID), FOREIGN KEY(Item_ID) References Inventory_Item(Item_ID))
#           ''')

In [6]:
# cursor.execute('''drop table Users''')
# cursor.execute('''drop table Inventory_Items''')
# cursor.execute('''drop table Orders''')

In [7]:
db.commit()
cursor.close()
db.close()

### utils file

In [8]:
ADMIN_ROLE_NAME = "Admin"
BUYER_ROLE_NAME = "Buyer"
SELLER_ROLE_NAME = "Seller"

ORDER_RECEIVED = "Order Received(Shipment Pending)"
ORDER_RETURN_INIT = "Return Initiated"

TEST_ADMIN_NAME = 'KellyZhang'
TEST_PASSWORD = '123'
TEST_FIRST_NAME = 'Kelly'
TEST_LAST_NAME = 'Zhang'
TEST_EMAIL = '123abc@def.com'
TEST_ADDRESS = '123 FASDF, AUSTIN, TX 78731'
TEST_PHONE_NUM = 1234567890
TEST_ID_NUM = 0

### activities.py

In [9]:
def register(user_name, password, user_role, first_name, last_name, email, address, phone_num) -> bool:

    if user_role.lower() == ADMIN_ROLE_NAME.lower():
        new_admin_user = Admin(user_name, password, first_name,
                               last_name, email, address, phone_num)
        write_user_into_database(ADMIN_ROLE_NAME, new_admin_user)
        print("Register succeeded! Returning to the main menu...\n")
        
    elif user_role.lower() == SELLER_ROLE_NAME.lower():
        new_seller_user = Seller(user_name, password, first_name,
                               last_name, email, address, phone_num)
        move_seller_to_pending(new_seller_user)
        print("Your registration is pending for authorization, thank you for your patience.")
        
    elif user_role.lower() == BUYER_ROLE_NAME.lower():
        print('This user is a buyer.')
        new_buyer_user = Buyer(user_name, password, first_name,
                                 last_name, email, address, phone_num)
        print('Writing the new buyer into db...')
        write_user_into_database(BUYER_ROLE_NAME, new_buyer_user)
        print("Register succeeded! Returning to the main menu...\n")
        
    else:
        print("Unknown role. Please try again.\n")
        return False

    return True

In [10]:
def check_duplicate(user_name):
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    try:
        cur_result = cursor.execute('''SELECT * FROM Users where User_Name = ? ''',(str(user_name),))
        db.commit()
        if len(cur_result.fetchall()) != 0:
            cursor.close()
            db.close()
            return True
        else:
            cursor.close()
            db.close()
            return False
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return True

In [11]:
def login(user_name, password):
    user_name = TEST_ADMIN_NAME
    password = TEST_PASSWORD
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cur_result = cursor.execute('''SELECT * FROM Users where User_Name = ? AND Password = ? ''',(user_name, password))
        db.commit()
        if len(cur_result.fetchall()) != 0:
            print("LogIn Succeed")
            cursor.close()
            db.close()
            return True
        else:
            print("Unable to LogIn")
            cursor.close()
            db.close()
            return False
        print("db is closed!")
        return True

    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False
    

In [12]:
def return_role(user_name):
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    
    try:
        cur_result = cursor.execute('''SELECT User_Type FROM Users where User_Name = ? ''', (str(user_name),))
        cur_result_all = cur_result.fetchall()
        db.commit()
        cursor.close()
        db.close()
        
        for row in cur_result_all:
            return row[0]
        
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        

In [13]:
def write_user_into_database(user_role, user) -> bool:
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cursor.execute('''Insert into Users(User_Name, Password, User_Type, First_Name, Last_Name, Email, Address, Phone_Num, User_ID) VALUES(?,?,?,?,?,?,?,?,?)''',
                       (user.user_name, user.password, user_role, user.first_name, user.last_name, user.email, user.address, user.phone_num, None))
        db.commit()
        cursor.close()
        db.close()
        print("db is closed!")
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

In [14]:
PENDING_SELLER_LIST = list()
def move_seller_to_pending(seller) -> bool:
    PENDING_SELLER_LIST.append(seller)
    return True

In [15]:
def get_userid_from_username(user_name):
    print("get_userid_by_username")
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cursor.execute('''SELECT User_ID FROM Users WHERE User_Name = ?''', (user_name,))
        db.commit()
        cursor.close()
        db.close()
        print("db is closed!")
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

### admin.py

In [16]:
class Admin:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num
#         self.id_num = id_num
    
    @staticmethod
    def authorize_sellers() -> bool:
        print("Hi Admin! Please check and authorize the following sellers for registration:")
        
        while (len(PENDING_SELLER_LIST) > 0):
            seller = PENDING_SELLER_LIST[0]
            print('PENDING_SELLER_LIST size: ' + str(len(PENDING_SELLER_LIST)) + "\n")
            # Notify the current admin about the seller's info
            print("Seller basic: \n", 
                  ((seller.user_name), (seller.password), (SELLER_ROLE_NAME), 
                   (seller.first_name), (seller.last_name), (seller.email), 
                   (seller.address), (seller.phone_num), (seller.id_num)))
            decision = input('Do you want to grant this seller permission to the system (Y/N)? ')
            if decision.lower() == 'y':
                print("Writing seller into database...")
                write_user_into_database(SELLER_ROLE_NAME, seller)
                print("Finished.\n")

            print("Removing the seller from the pending list...")
            PENDING_SELLER_LIST.remove(seller)
            print('PENDING_SELLER_LIST size: ' + str(len(PENDING_SELLER_LIST)) + "\n")
            
        print("Back to Admin Menu...\n")
        return True

    @staticmethod
    # Remove a user from the user table if needed
    def remove_user() -> bool:
        answer = input("Please input the user name you want to remove: ")
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''DELETE FROM Users 
                              WHERE User_Name = ?''',(answer,))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

        return False

### seller.py

In [17]:
class Seller:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num
    
    image_dir = "Images/"
    export_dir = "Exports/"
    import_dir = "ImportFiles/"
        
    @staticmethod
    def add_item(Brand_Name, Cloth_Type, Size, Gender, Original_Price, 
                 Rental_Price, Owner_ID, Location,
                 Cloth_Image, Available_From) :
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        
        blobData = None

        for image in glob.glob(Seller.image_dir+'/*'):
            if (str(image).replace(Seller.image_dir,'')) == Cloth_Image:
                with open(image, 'rb') as file:
                    blobData = file.read()
        try:
            cursor.execute('''INSERT INTO Inventory_Items(Brand_Name, Cloth_Type, Size, 
                                Gender, Original_Price, Rental_Price, Owner_ID, Location, 
                                Cloth_Image, Available_From) VALUES(?,?,?,?,?,?,?,?,?,?)''',
                           (Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, Owner_ID, Location, blobData, Available_From))
            db.commit()
            cursor.close()
            db.close()
            return True

        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

        
    @staticmethod
    def remove_item(Item_ID):

        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''Delete from Inventory_Items where Item_ID == ?''',(str(Item_ID)))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

    @staticmethod
    def update_item(Item_ID, Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, Owner_ID, Location, Cloth_Image, Available_From):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()

        for image in glob.glob(Seller.image_dir + '/*'):
            if (str(image).replace(Seller.image_dir, '')) == Cloth_Image:
                with open(image, 'rb') as file:
                    blobData = file.read()
        
        try:
            cursor.execute('''UPDATE Inventory_Items SET Brand_Name= ?, Cloth_Type = ?, Size = ?, Gender = ?, 
                            Original_Price =?, Rental_Price =?, Owner_ID=?, Location=?, Cloth_Image=?,
                            Available_From=? WHERE Item_ID = ?''', 
                           (Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, Owner_ID, 
                            Location, blobData, Available_From, Item_ID))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

    @staticmethod
    def ship_item(Order_ID):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''UPDATE Orders SET Order_Status = 'Order Shipped', Shipping_Date = ?
                              WHERE Order_ID = ?''', (date.today(), Order_ID))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

    @staticmethod
    def refund_deposit(Order_ID):
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Update Orders 
                    set Order_Status = 'Return Received(Deposit Refunded)'
                    where Order_ID = ? and Return_Date >= ? and Order_Status = "Return Initiated" ''', (str(Order_ID),date.today()))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def withhold_deposit(Order_ID):
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Update Orders 
                        set Order_Status = 'Return Not Received(Deposit Withheld)'
                        where Order_ID = ? and Return_Date < ? and Order_Status != "Return Initiated" ''', (str(Order_ID),date.today()))
        db.commit()
        Database.Database.close_connection()

### buyer.py

In [30]:
class Buyer:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num
#         self.id_num = id_num

    @staticmethod    
    def view_item():
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''SELECT * FROM Inventory_Items''')
            result = cursor.fetchall()
            print("Inventory size: " + str(len(result)))
            for item in result:
                print("Item_ID: " + str(item[0]) + "; Brand_Name: " + str(item[1]) + "; Cloth_Type: " + str(item[2]) + 
                      "; Size: " + str(item[3]) + "; Gender: " + str(item[4]) + "; Original_Price: " + str(item[5]) + 
                      "; Rental_Price: " + str(item[6]) + "; Available_From: " + str(item[7]) + 
                      "; Owner_ID: " + str(item[8]) + "; Location: " + str(item[9]) + ".\n")
            print("\n")
            
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return
        
        cursor.close()
        db.close()
    
    @staticmethod
    def rent_item(id_num) -> bool:
        print("Renting item")
        item_id_num = input("Please input the item num you want to rent: ")
        shipping_address = input("Please input your shipping address: ")
        rent_days = int(input("Please input how many days you want to rent the item: "))
        delivery_date = date.today() + timedelta(days=7)
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''INSERT INTO Orders(Order_ID, User_ID, Item_ID, Delivery_Date, Return_Date, Shipping_Address, Order_Status, Shipping_Date)
                              VALUES(?,?,?,?,?,?,?,?)
                           ''',(None, id_num, item_id_num, delivery_date, delivery_date + timedelta(days = rent_days), shipping_address, ORDER_RECEIVED, ""))
#             # update Available_From in Inventory_Items table 
#             cursor.execute('''UPDATE Inventory_Items
#                               WHERE Available_From = ? ''', (shipping_date + timedelta(delivery_date) + timedelta(days = rent_days)))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
    
    @staticmethod
    def return_item() -> bool:
        print("Returning item")
        item_id_num = input("Please input the item num you want to return: ")
        # update Order_Status in Orders table
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''UPDATE Orders SET Order_Status = ? WHERE Item_ID = ?''', (ORDER_RETURN_INIT, item_id_num,))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
        return False
    

### main.py

In [19]:
def show_main_menu() -> bool:
    print("Hi there! Welcome to Cloth on Fly! What's Next?")
    print("[1] Register")
    print("[2] Log In")
    print("[3] Quit")
    main_answer = input("Input the action number here: ")
    # registration
    if main_answer == "1":
        print("Please input the following information to register!")
        
        user_name = input("Username: ")
        
        is_user_name_duplicate = check_duplicate(user_name)
        if (is_user_name_duplicate):
            print("This user name already exists in the database. Please use a different one for registration\n")
            return True
        
        password = input("Password: ")
        user_role = input("Do you want to register as admin/seller/buyer: ")
        first_name = input("First Name: ")
        last_name = input("Last Name: ")
        email = input("Email: ")
        address = input("Address: ")
        phone_num = int(input("Phone Number: "))
        
        register(user_name, password, user_role, first_name, last_name, email, address, phone_num)
        return True

    # login
    elif main_answer == "2":
        print("Hey Flyer! Please input your user name and password to log in: ")
        user_name = input("Username: ")
        password = input("Password: ")
        user_role = return_role(user_name)
        if user_role == None:
            print("User does not exist. Please try again.\n")
        else:
            print("User role is: " + user_role + "\n")
            user_menu_keep_looping = True
            while (user_menu_keep_looping):
                user_menu_keep_looping = show_user_menu(user_name, user_role)
        return True
    
    # quit
    elif main_answer == "3":
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [20]:
def show_user_menu(user_name, user_role):
    if (user_role == ADMIN_ROLE_NAME):
        return show_admin_menu(user_name)
    elif (user_role == SELLER_ROLE_NAME):
        return show_seller_menu(user_name)
    else:
        return show_buyer_menu(user_name)

In [21]:
def show_admin_menu(user_name):
    print(">>>>>>>> Admin Menu\n")
    print("[1] Authorize Sellers")
    print("[2] Remove An User")
    print("[3] Add An Item")
    print("[4] Remove An Item")
    print("[5] Update An Item")
    print("[6] Log Out")
    user_answer = input("Input the action number here: ")
    
    user_id = get_userid_from_username(user_name)
    
    if user_answer == '1':
        print("Retrieving pending sellers list...\n")
        Admin.authorize_sellers()
        print("Finished pending sellers review.\n")
        return True
    elif user_answer == '2':
        print("Removing an user...\n")
        Admin.remove_user()
        print("Finished user deletion.\n")
        return True
    elif user_answer == '3':
        print("Adding an item...\n")
        Brand_Name = input("Please input the brand of the cloth: ")
        Cloth_Type = input("Please input the type of the cloth: ")
        Size = input("Please input the size of the cloth: ")
        Gender = input("Please input the gender that fit for the cloth: ")
        Original_Price = input("Please input the original price of the cloth: ")
        Rental_Price = input("Please input the rental per day price for the cloth: ")
        Location = input("Please input the current location of the cloth: ")
        Cloth_Image = input("Please input the name of your cloth image")
#         Deposit = input("Please input the deposit you want to hold for the cloth: ")
        Seller.add_item(Brand_Name, Cloth_Type, Size, Gender, Original_Price, 
                         Rental_Price, user_id, Location, Cloth_Image, Available_From = date.today())
        print("Finished adding the item.\n")
        return True
    elif user_answer == '4':
        Item_ID = input("Please input the item ID that you want to remove: ")
        print("Removing an item...\n")
        Seller.remove_item(Item_ID)
        print("Finished removing the item.\n")
        return True
    elif user_answer == '5':
        print("Updating an item...\n")
        Item_ID = input("Please input the item ID that you want to update: ")
        Brand_Name = input("Please update the brand of the cloth: ")
        Cloth_Type = input("Please update the type of the cloth: ")
        Size = input("Please update the size of the cloth: ")
        Gender = input("Please update the gender that fit for the cloth: ")
        Original_Price = input("Please update the original price of the cloth: ")
        Rental_Price = input("Please update the rental per day price for the cloth: ")
        Owner_ID = input("Please update the owner ID: ")
        Location = input("Please update the current location of the cloth: ")
        Cloth_Image = input("Please update the name of your cloth image: ")
        Available_From = input("Please update the availablity date using the format xxxx-xx-xx: ")
        Seller.update_item(Item_ID, Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, Owner_ID, Location, Cloth_Image, Available_From)
        print("Finished updating the item.\n")
        return True
    elif user_answer == '6':
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [22]:
def show_seller_menu(user_name):
    print(">>>>>>>> Seller Menu\n")
    print("[1] Add An Item")
    print("[2] Remove An Item")
    print("[3] Update An Item")
    print("[4] Ship An Item")
    print("[5] Refund Deposit")
    print("[6] Withhold Deposit")
    print("[7] Log Out")
    user_answer = input("Input the action number here: ")
    
    user_id = get_userid_from_username(user_name)

    if user_answer == '1':
        print("Adding an item...\n")
        Brand_Name = input("Please input the brand of the cloth: ")
        Cloth_Type = input("Please input the type of the cloth: ")
        Size = input("Please input the size of the cloth: ")
        Gender = input("Please input the gender that fit for the cloth: ")
        Original_Price = input("Please input the original price of the cloth: ")
        Rental_Price = input("Please input the rental per day price for the cloth: ")
        Location = input("Please input the current location of the cloth: ")
        Cloth_Image = input("Please input the name of your cloth image")
#         Deposit = input("Please input the deposit you want to hold for the cloth: ")
        Seller.add_item(Brand_Name, Cloth_Type, Size, Gender, Original_Price, 
                         Rental_Price, user_id, Location, Cloth_Image, Available_From = date.today())
        print("Finished adding the item.\n")
        return True
    elif user_answer == '2':
        Item_ID = input("Please input the item ID that you want to remove: ")
        print("Removing an item...\n")
        Seller.remove_item(Item_ID)
        print("Finished removing the item.\n")
        return True
    elif user_answer == '3':
        print("Updating an item...\n")
        Item_ID = input("Please input the item ID that you want to update: ")
        Brand_Name = input("Please update the brand of the cloth: ")
        Cloth_Type = input("Please update the type of the cloth: ")
        Size = input("Please update the size of the cloth: ")
        Gender = input("Please update the gender that fit for the cloth: ")
        Original_Price = input("Please update the original price of the cloth: ")
        Rental_Price = input("Please update the rental per day price for the cloth: ")
        Owner_ID = input("Please update the owner ID: ")
        Location = input("Please update the current location of the cloth: ")
        Cloth_Image = input("Please update the name of your cloth image: ")
        Available_From = input("Please update the availablity date using the format xxxx-xx-xx: ")
        Seller.update_item(Item_ID, Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, Owner_ID, Location, Cloth_Image, Available_From)
        print("Finished updating the item.\n")
        return True
    elif user_answer == '4':
        print("Shipping an item...\n")
        Order_ID = input("Please input your order ID that you shipped out: ")
        Seller.ship_item(Order_ID)
        return True
    elif user_answer == '5':
        print("Deposit refund processing...\n")
        return True
    elif user_answer == '6':
        print("Deposit withholding...\n")
        return True
    elif user_answer == '7':
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [28]:
def show_buyer_menu(user_name):
    print(">>>>>>>> Buyer Menu\n")
    print("[1] View Items")
    print("[2] Rent An Item")
    print("[3] Return An Item")
    print("[4] Log Out")
    user_answer = input("Input the action number here: ")

    if user_answer == '1':
        print("Checking inventory...\n")
        Buyer.view_item()
        return True
    elif user_answer == '2':
        print("Placing an order...\n")
        user_id = get_userid_from_username(user_name)
        Buyer.rent_item(user_id)
        print("Complete the order!\n")
        return True
    elif user_answer == '3':
        print("Item return processing...\n")
        Buyer.return_item()
        print("Return Initiated\n")
        return True
    elif user_answer == '4':
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True
    

In [31]:
def main():
    keep_looping = True
    while (keep_looping):
        keep_looping = show_main_menu()
    
if __name__ == '__main__':
    main()

Hi there! Welcome to Cloth on Fly! What's Next?
[1] Register
[2] Log In
[3] Quit
Input the action number here: 2
Hey Flyer! Please input your user name and password to log in: 
Username: ke
Password: ke
User role is: Buyer

>>>>>>>> Buyer Menu

[1] View Items
[2] Rent An Item
[3] Return An Item
[4] Log Out
Input the action number here: 1
Checking inventory...

Inventory size: 4
Item_ID: 2; Brand_Name: Levis; Cloth_Type: Jeans; Size: L; Gender: Male; Original_Price: 59.99; Rental_Price: 0.99; Available_From: 2019-09-08; Owner_ID: 2; Location: Austin.

Item_ID: 3; Brand_Name: HM; Cloth_Type: TShirt; Size: M; Gender: Uni; Original_Price: 19.99; Rental_Price: 3.99; Available_From: 2019-07-27; Owner_ID: 1; Location: Austin, TX.

Item_ID: 5; Brand_Name: Zara; Cloth_Type: Sweater; Size: L; Gender: Female; Original_Price: 99.0; Rental_Price: 5.0; Available_From: 2019-07-27; Owner_ID: 1; Location: Austin.

Item_ID: 6; Brand_Name: BOSS; Cloth_Type: Suits; Size: M; Gender: Male; Original_Price: 2